In [0]:
# Import dependencies
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy import extract 
import json

In [0]:
# Setup the DB
engine = create_engine("sqlite:///hawaii.sqlite")


In [0]:
Base = automap_base()

In [0]:
# Reflect tables
Base.prepare(engine, reflect=True)

In [0]:
# We’ll create a variable for each of the classes to reference them later
measurement = Base.classes.measurement
station = Base.classes.station


In [0]:
# Create link from python to our DB
session = Session(engine)


In [0]:
# Flask app
app = Flask(__name__)

### Notice that the last route in the welcome route below "/api/v1.0/month_stats/month" is the route I created for the challenge.

In [0]:
# Welcome route
@app.route("/")
def welcome():
	return( '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    /api/v1.0/month_stats/month
    ''')



In [0]:
# Percipitation route
@app.route("/api/v1.0/precipitation")
def precipitation():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    precipitation = session.query(measurement.date, measurement.prcp).\
	filter(measurement.date >= prev_year).all()
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)


In [0]:
# Stations Route
@app.route("/api/v1.0/stations")
def stations():
    results = session.query(station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations)


In [0]:
# Monthly Temperature route
@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(measurement.tobs).\
    filter(measurement.station == 'USC00519281').\
    filter(measurement.date >= prev_year).all()
    temps = list(np.ravel(results))
    return jsonify(temps)


In [0]:
@app.route("/api/v1.0/temp/<start>/<end>")
def stats(start, end):
    session = Session(engine)
    results = session.query(func.min(measurement.tobs),func.avg(measurement.tobs),func.max(measurement.tobs)).\
    filter(measurement.date >= start).filter(measurement.date <= end).all()
    temps = list(np.ravel(results))
    session.close()
    return jsonify(temps)

# Challenge Part:
### Run it and enter a number from 1 to 12 instead of the month in the route, i.e. "/api/v1.0/month_stats/6", this should give you the stats for the month of June.

In [0]:
#Month Stats Route using .describe() function
@app.route("/api/v1.0/month_stats/<month>")
def month_stats(month):
    session = Session(engine)
    stats = []
    stats= session.query(measurement.date, measurement.prcp, measurement.tobs, measurement.station).filter(extract('month', measurement.date)==month).all()
    df_stats = pd.DataFrame(stats, columns=['date','precipitation','tobs','station'])
    df_stats.set_index(df_stats['date'], inplace=True)
    df_stats = df_stats.sort_index()
    stats_month = df_stats.describe()
    session.close
    return stats_month.to_json()


#### December Stats
{
precipitation: {
count: 1405,
mean: 0.2168185053,
std: 0.5413993941,
min: 0,
25%: 0,
50%: 0.03,
75%: 0.15,
max: 6.42
},

tobs: {
count: 1517,
mean: 71.0415293342,
std: 3.7459198253,
min: 56,
25%: 69,
50%: 71,
75%: 74,
max: 83
}
}

#### June Stats
{
precipitation: {
count: 1574,
mean: 0.1363595934,
std: 0.3357312525,
min: 0,
25%: 0,
50%: 0.02,
75%: 0.12,
max: 4.43
},

tobs: {
count: 1700,
mean: 74.9441176471,
std: 3.2574170517,
min: 64,
25%: 73,
50%: 75,
75%: 77,
max: 85
}
}

## Analysis and Findings:

1) The mean of the precipitation for December is 57% higher than the mean in precipitation in June.  This makes sense since it rains less in summer than in winter months. 

2) Both the data for the precipitation in June and December are a bit skewed since the median is much lower than the mean in both months. Also, there is a great variance in the precipitation data for both December and June, this can be deduced from the standard deviation.  

3) The mean of the temperature for June is 6% higher than the mean of temparature in December.  Its obvious that the average temparatures in June will be a bit higher than December since the weather is usually warmer in June than December.

4) Looking at the temparature data in both December and June we can see that the median tempratures for both months compared to their means is almost the same which means that there will be no skewness, if its plotted in a histogram. There isn't a big variance in the temperature data for both June and December. 

## Recommendations:

1) In order to conduct a fair weather comparison and analysis between the months of December and June we should take a look at the June and December data, and try to do some clean up, by excluding outliers, or duplicates etc.

2) Also, maybe instead of just looking at December vs. June weather, we can analyze the weather for the winter months vs. the summer months for example (December, January, and February) Vs. (June, July, and August). In this case we will need to update the code above.

3) Also we can get the same statistical data for each month per station and compare and analyze how it is different from a station to a station. 
